In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb
from data_import import preprocess_import_data

In [5]:
df = pd.concat([pd.read_excel('./scan_data/202109_all_scan_data.xlsx'), 
                pd.read_excel('./scan_data/202110_all_scan_data.xlsx'),
                pd.read_excel('./scan_data/202111_all_scan_data.xlsx'),
                pd.read_excel('./scan_data/202112_all_scan_data.xlsx')])

preprocess_import_data(df)
df.head(3)

,age,gender,height_cm,weight_kg,adult_child,department,hospital_ward,scan_area,scan_method,scan_protocol,scan_series,kV,mA,rotation_time,CTDI
0,72,M,170.0,83.0,成人,救急科,外来,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,1: Plain,100.0,366.41,0.5,16.64
1,72,M,170.0,83.0,成人,救急科,外来,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,2: CE,100.0,366.41,0.5,16.61
2,85,M,171.0,58.9,成人,循環器内科,外来,胸部〜骨盤CT,単純,5.7 P+CE Chest-Pelvis Routine,1: Plain,120.0,234.59,0.5,16.66


In [6]:
oe = OrdinalEncoder()
oe.set_output(transform='pandas')
cat_cols = df.select_dtypes(exclude=np.number).columns.to_list()
df[cat_cols] = oe.fit_transform(df[cat_cols])

df.dropna(inplace=True)

target = 'CTDI'

X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
lgb_reg = lgb.LGBMRegressor(learning_rate=0.01, random_state=0)
lgb_reg.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.01, random_state=0)

In [8]:
y_pred = lgb_reg.predict(X_test)

In [9]:
print(f'mean_absolte_error: {mean_absolute_error(y_test, y_pred)}')
print(f'mean_absolute_percentage_error: {mean_absolute_percentage_error(y_test, y_pred)}')

mean_absolte_error: 4.196680328883509
mean_absolute_percentage_error: 0.3393001027654956


In [10]:
df_predict = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

df_predict['diff'] = df_predict['y_test'] - df_predict['y_pred']

df_predict

,y_test,y_pred,diff
898,18.68,16.245965,2.434035
1645,27.50,24.362623,3.137377
1918,19.13,19.343448,-0.213448
104,15.44,15.840165,-0.400165
2367,21.86,22.771102,-0.911102
...,...,...,...
1841,34.38,29.432585,4.947415
538,10.22,13.354047,-3.134047
2239,11.46,13.301820,-1.841820
1888,6.60,10.848589,-4.248589
